In [1]:
import pandas as pd
import numpy as np
import PySimpleGUI as sg
import os
import time
from txdpy import get_letter,get_num
import win32api,win32con
from datetime import datetime
import re
import xlwings as xw
import math

In [2]:
# 读取excel数据
def read_data(file_path,first_cell):
    app = xw.App(visible=False, add_book=False)
    wb = app.books.open(file_path) # 打开Excel文件
    sheet = wb.sheets[0] # 选择第0个表单
    data = sheet.range(first_cell).expand().value # 读取全表数据
    cols = data[0]
    df = pd.DataFrame(data[1:], columns=cols)
    return df

In [3]:
root_path = sg.popup_get_folder("请选择根目录文件夹")
print(root_path)

D:/DeskTop/Learn/6-Liz


In [4]:
listing_path = root_path + '/listing'

In [5]:
start_time = time.time()

In [6]:
listing_path

'D:/DeskTop/Learn/6-Liz/listing'

## 1.0 读取数据

In [7]:
# 读取新品在途进度表
old_time = time.time()
for file in os.listdir(root_path):
    if '$' not in file and '在途' in file and '进度表' in file and '结果' not in file:
        progress_transit_path = root_path + '/' + file
        print(progress_transit_path)
        df_progress_transit = pd.read_excel(progress_transit_path,sheet_name="各账号上架情况跟踪表-在途")
        print("size：{}".format(len(df_progress_transit)))
print("读取时间为%.2f秒钟"%(time.time()-old_time))

D:/DeskTop/Learn/6-Liz/【新品专项】在途新品模板进度表_2023-8-21.xlsx
size：10997
读取时间为1.78秒钟


In [8]:
# 读取MRP替换料总表
old_time = time.time()
for file in os.listdir(root_path):
    if '$' not in file and '替代ok' in file and '_' in file:
        altersku_mrp_path = root_path + '/' + file
        print(altersku_mrp_path)
        df_altersku_mrp = pd.read_excel(altersku_mrp_path,header=0)[['主料SKU','替换料1','替换料2','替换料3','替换料4','替换料5','替换料6']]
        df_altersku_mrp['sku_6'] = np.nan
        print("size：{}".format(len(df_altersku_mrp)))
print("读取时间为%.2f秒钟"%(time.time()-old_time))

D:/DeskTop/Learn/6-Liz/替代ok表_20230823_172141.xlsx
size：40951
读取时间为1.87秒钟


In [9]:
# 读取listing
list_listing = []
old_time = time.time()
for file in os.listdir(listing_path):
    file_path = listing_path + '/' + file
    if "$" not in file:
        df = read_data(file_path,'A2')
        df = df[['ItemID', 'Seller', '主SKU', 'SKU',  '刊登方式(天)', '在线数量','刊登日期']]
        print("====== {}——读取完成 ======".format(file_path))
        list_listing.append(df)
# 合并
df_listing = pd.concat(list_listing)
print("读取时间为%.2f秒钟"%(time.time()-old_time))

====== D:/DeskTop/Learn/6-Liz/listing/2U-Listing20230818.xlsx——读取完成 ======
====== D:/DeskTop/Learn/6-Liz/listing/AP-Listing20230818.083620.xlsx——读取完成 ======
====== D:/DeskTop/Learn/6-Liz/listing/US-T.xlsx——读取完成 ======
====== D:/DeskTop/Learn/6-Liz/listing/大账号-Listing20230818.083044.xlsx——读取完成 ======
读取时间为47.80秒钟


In [10]:
# seller对应不同的账号名称，分别为：
# 2U: partswholesale2u
# Ap: a-premium-us
# US-T: total_autopart
# 大账号: autoparts_premium
# partswholesale2u,a-premium-us,total_autopart,autoparts_premium
df_listing.head()

,ItemID,Seller,主SKU,SKU,刊登方式(天),在线数量,刊登日期
0,2.841186e+11,partswholesale2u,P-USDLAAC030,P-USDLAAC030,一口价(GTC),25.0,2020-12-18 09:28:26
1,2.841186e+11,partswholesale2u,P-USDLA81410,P-USDLA81410,一口价(GTC),25.0,2020-12-18 09:27:33
2,2.841186e+11,partswholesale2u,"P-USDLA3S020(质量客诉,需退回检测)","P-USDLA3S020(质量客诉,需退回检测)",一口价(GTC),25.0,2020-12-18 09:26:25
3,2.841186e+11,partswholesale2u,"P-USDLA3S010+USDLA3S020(质量客诉,需退回检测)","P-USDLA3S010+USDLA3S020(质量客诉,需退回检测)",一口价(GTC),25.0,2020-12-18 09:25:58
4,2.841186e+11,partswholesale2u,P-USDLA3S010,P-USDLA3S010,一口价(GTC),25.0,2020-12-18 09:25:51


## 2.0 listiing处理

In [11]:
df_listing = df_listing.reset_index(drop=True)
df_listing = df_listing.drop_duplicates()
df_listing['ItemID'] = df_listing['ItemID'].apply(int)
'size：{}'.format(len(df_listing))

'size：266268'

In [12]:
list_sku=df_listing['主SKU'].to_list()

In [13]:
# 主SKU清洗
def clear_sku(list_skux):
    for i in range(0,len(list_skux)):
        a = re.sub('\（','(',list_skux[i])#转化(
        a1 = re.sub('\）',')',a)#转化）
        a2 = re.sub('\(.*?\)','',a1)#删除（）内信息
        a2 = re.sub('-测试','',a2)
        a2 = re.sub('，','',a2)
        a3 = re.sub(r'^[A-Z]{1,2}-{1}','',a2,count=1)#删除前缀
        a4 = re.sub('-NC','',a3)#删除-NC后缀-少数，USOS24423H-NC，USIC02680I-6-NC(质量投诉，需退回检测)
        a5 = re.sub('-N$','',a4)#删除-N后缀
        y=re.sub('-Scion$','',a5)
        z=re.sub(r'-Chevrolet$','',y)
        h = re.sub('-1\*2','*2',z)
        j =re.sub('USIC20181/USIC65500','USIC65500',h)
        k = re.sub('USBCB92767/USBC477563','USBC477563',j)
        l = re.sub('USHLS29918/USHLS60843','USHLS60843',k)
        m = re.sub('USGS436403','USGS43640B',l)
        m1=re.sub(r'USDS36847case停售10.19','USDS36847',m)
        n = re.sub('-US1$','',m1)
        o = re.sub('-Ford$','',n)
        p = re.sub('-MAP$','',o)
        q = re.sub('-Dodge$','',p)
        a6 = re.sub('-2021','',q)#删除后缀为-2021的SKU，USSGTE003-2021，USSGTE004-2021
        z=re.sub('case较高暂停出售','',a6)
        a7 = re.sub('-U$','',z)#删除后缀为-U的SKU，USGS47710-U，USIM15178-U先发测试
        a8 = re.sub('-4Runner$','',a7)#删除后缀为-4runner的SKU，USIFTO001Z-4Runner
        a9 = re.sub('-1A$','',a8)#删除后缀为‘-1A’的SKU,'-1AL'无误
        a10 = re.sub('-1B$','',a9)#删除后缀为‘-1B’的SKU
        a11 = re.sub('-1C$','',a10)#删除后缀为‘-1C’的SKU,'-1CA'无误
        a12 = re.sub('-1D$','',a11)#删除后缀为‘-1D’的SKU
        a13 = re.sub('-C1$','',a12)#删除后缀为‘-C1’的SKU
        a14 = re.sub('-M$','',a13)#删除后缀为‘-M’的SKU
        a15 = re.sub('-A$','',a14)#删除后缀为‘-A’的SKU
        a16 = re.sub('-B$','',a15)#删除后缀为‘-B’的SKU
        a17 = re.sub('-R2021$','-R',a16)
        a18 = re.sub('-L2021$','-L',a17)#删除后缀‘2021’
        a19= re.sub(' / ','/',a18)
        a20=re.sub('-$','',a19)
        a21=re.sub('-Rear$','',a20)
        a22=re.sub('-1.7kw$','',a21)
        a23=re.sub('-GMC$','',a22)
        a24=re.sub('-AMC$','',a23)
        a25=re.sub('-Chrysler$','',a24)
        a26=re.sub('-JPN$','',a25)
        a27=re.sub('-Other$','',a26)
        a28=re.sub('-US2$','',a27)
        a29=re.sub('-UR$','',a28)
        a29=re.sub('-UK$','',a29)
        a30=re.sub('-IT','',a29)
        a30=re.sub('-Ford$','',a30)
        a30=re.sub('-Left$','',a30)
        a30=re.sub('-Chevrolet$','',a30)
        a30=re.sub('-Dodge$','',a30)
        a30=re.sub('-N$','',a30)
        a30=re.sub('-D$','',a30)
        a32=re.sub('-DE$','',a30)
        a34=re.sub('-GMC1$','',a32)
        a35=re.sub('-GMC22$','',a34)
        a36=re.sub('-GMC2$','',a35)
        a37=re.sub('-Dodge Other$','',a36)
        a38=re.sub('-Dodge Ram$','',a37)
        a39=re.sub('-Oldsmobile$','',a38)
        a40=re.sub('-Reservoir$','',a39)
        a41=re.sub(r'-Chevy[0-9]{0,1}$','',a40)
        a42=re.sub(r'-UP$','',a41)
        a43=re.sub(r'-Return$','',a42)
        a44=re.sub(r'-MF$','',a43)
        a45=re.sub(r'-Front$','',a44)
        a46=re.sub(r'-Nissan$','',a45)
        a46=re.sub(r'-Toyota$','',a46)
        a46=re.sub(r'-VW$','',a46)
        a46=re.sub(r'-RH$','',a46)
        a46=re.sub(r'pin','',a46)
        a46=re.sub(r'-Case','',a46)
        a46=re.sub(r'-Saab','',a46)
        a46=re.sub(r'Note','',a46)
        a46=re.sub(r'\)','',a46)
        a46 = re.sub(r'[\u4e00-\u9fa5]', '', a46)
        a46 = re.sub(r'\-C','',a46)
        a46 = re.sub(r'\*','-',a46)
        a46=a46.rstrip('/')
        a46=a46.rstrip('+')
        a46=a46.strip()
        a46=a46.split('(')[0]
        list_skux[i]=a46
    return list_skux

In [14]:
list_sku = clear_sku(list_sku)
df_listing['处理后SKU'] = list_sku

In [15]:
# 处理异常UUS的SKU
index_u = df_listing[df_listing['处理后SKU'].str.startswith('UUS')].index.tolist()
for i in index_u:
    df_listing.loc[i,'处理后SKU'] = df_listing.loc[i,'处理后SKU'][1:]
    list_sku[i] = list_sku[i][1:]

In [16]:
# MRP替换料
df_altersku_mrp_alt =  df_altersku_mrp
df_altersku_mrp_alt = df_altersku_mrp_alt.copy()
df_altersku_mrp_alt['替换料SKU'] = df_altersku_mrp_alt.apply(lambda x: ','.join(x[x.notnull()]).split(','),axis=1)
df_altersku_mrp_alt_e = df_altersku_mrp_alt[['替换料SKU']].explode('替换料SKU')
list_altersku_mrp = df_altersku_mrp_alt_e['替换料SKU'].tolist()

In [17]:
# 给替代料增加标识符row
index_list = list(df_altersku_mrp_alt_e.index)
index_list = [str(x+1) for x in index_list]
df_altersku_mrp_alt_e['row'] = index_list
df_altersku_mrp_alt_e.columns = ['SKU', 'row']
df_altersku_mrp_alt_e = df_altersku_mrp_alt_e.drop_duplicates(keep='first')
# df_altersku_mrp_alt_e

# 3.0 在途表匹配对应账号刊登时间

## 3.1 获取对应账号listing

In [18]:
# 按照'seller'分组，对每个分组应用函数进行去重
def keep_earliest_time(group):
    return group.sort_values(by='刊登日期').drop_duplicates(subset='row', keep='first')

In [19]:
# 筛选listing需要的列数
df_listing_tmp = df_listing
df_listing_tmp = df_listing_tmp[['Seller','处理后SKU','刊登日期']]
df_listing_tmp.columns = ['Seller','SKU', '刊登日期']
df_listing_tmp = df_listing_tmp.copy()
df_listing_tmp['刊登日期'] = pd.to_datetime(df_listing_tmp['刊登日期'], errors='coerce')

In [20]:
# listing的SKU匹配到对应替代料的row
df_listing_tmp = pd.merge(left=df_listing_tmp, right=df_altersku_mrp_alt_e, how='left', left_on='SKU', right_on='SKU').fillna('')
df_listing_tmp = df_listing_tmp.groupby('Seller').apply(keep_earliest_time).reset_index(drop=True)
df_listing_tmp = df_listing_tmp.reset_index(drop=True)
df_listing_tmp.head()

,Seller,SKU,刊登日期,row
0,a-premium-us,USGS63320,2017-04-02 15:04:17,22792
1,a-premium-us,USGS41590,2017-04-02 15:04:22,21824
2,a-premium-us,USGS61780,2017-04-02 15:04:27,22739
3,a-premium-us,USGS65010,2017-04-02 15:04:48,22842
4,a-premium-us,USGS43790,2017-04-02 15:04:53,22611


In [21]:
# 选择ap账号的listing
df_listing_ap = df_listing_tmp[df_listing_tmp['Seller']=='a-premium-us']
df_listing_2u = df_listing_tmp[df_listing_tmp['Seller']=='partswholesale2u']
df_listing_total = df_listing_tmp[df_listing_tmp['Seller']=='total_autopart']

## 3.2在途表匹配对应账号信息

In [22]:
# 在途表标记原始行号
df_progress_transit['原始行号'] = [int(i+2) for i in range(df_progress_transit.shape[0])]
df_progress_transit_tmp_total = df_progress_transit
df_progress_transit_tmp_total = pd.merge(left=df_progress_transit_tmp_total, right=df_altersku_mrp_alt_e, how='left', left_on='SKU', right_on='SKU')

In [23]:
def convert_to_datetime(value):
    if isinstance(value, str):
        try:
            return datetime.strptime(value, '%Y/%m/%d')
        except ValueError:
            return value  # 如果无法转换为日期时间格式，保持原值
    elif pd.isna(value):  # 如果是 NaT，则保持原值
        return pd.NaT
    else:
        return value  # 如果已经是日期时间格式，保持原值

In [24]:
df_progress_transit_tmp_total['大账号'] = df_progress_transit_tmp_total['大账号'].apply(convert_to_datetime)
df_progress_transit_tmp_total['ap'] = df_progress_transit_tmp_total['ap'].apply(convert_to_datetime)
df_progress_transit_tmp_total['2u'] = df_progress_transit_tmp_total['2u'].apply(convert_to_datetime)
df_progress_transit_tmp_total['total'] = df_progress_transit_tmp_total['total'].apply(convert_to_datetime)

In [25]:
# 选择大账号不为空
df_progress_transit_tmp = df_progress_transit_tmp_total[pd.notna(df_progress_transit_tmp_total['大账号'])].reset_index(drop=True)

# 选择大账号不为空，ap账号为空的值
df_ap = df_progress_transit_tmp_total[pd.isna(df_progress_transit_tmp_total['ap'])].reset_index(drop=True).fillna('')

# 选择大账号不为空，2u账号为空的值
df_2u = df_progress_transit_tmp_total[(pd.isna(df_progress_transit_tmp_total['2u']))].reset_index(drop=True).fillna('')

# 选择大账号不为空，total账号为空的值
df_total = df_progress_transit_tmp_total[(pd.isna(df_progress_transit_tmp_total['total']))].reset_index(drop=True).fillna('')

In [26]:
# df_progress_transit_tmp

### 3.3 在途匹配Ap账号信息

In [27]:
# 选择有row值来匹配相关替代料，无row值的则为没有替代料，后续直接根据SKU匹配刊登时间
df_ap_row = df_ap[df_ap['row']!=''].reset_index(drop=True)
df_ap_row = pd.merge(left=df_ap_row, 
                     right=df_listing_ap, 
                     how='left', 
                     left_on='row', 
                     right_on='row', 
                     suffixes=('','_x'))
df_ap_row['ap'] = [pd.NaT if pd.isna(df_ap_row.loc[i,'刊登日期']) else df_ap_row.loc[i,'刊登日期'] for i in range(df_ap_row.shape[0])]
df_ap_row = df_ap_row[df_ap_row.columns[:-3]]
# df_ap_row

In [28]:
# 无row值的则为没有替代料，直接根据SKU匹配刊登时间
df_ap_notrow = df_ap[df_ap['row']==''].reset_index(drop=True)
df_ap_notrow = pd.merge(left=df_ap_notrow, 
                        right=df_listing_ap, 
                        how='left', 
                        left_on='SKU',
                        right_on='SKU', 
                        suffixes=('','_x'))
df_ap_notrow['ap'] = [df_ap_notrow.loc[i,'ap'] if pd.isna(df_ap_notrow.loc[i,'刊登日期']) else df_ap_notrow.loc[i,'刊登日期'] for i in range(df_ap_notrow.shape[0])]
df_ap_notrow = df_ap_notrow[df_ap_notrow.columns[:-3]]
# df_ap_notrow

In [29]:
df_ap_tmp = pd.concat([df_ap_row,df_ap_notrow])
df_ap_tmp = df_ap_tmp.sort_values(by='原始行号').reset_index(drop=True)
# df_ap_tmp

### 3.4 在途匹配2u账号信息

In [30]:
# 选择有row值来匹配相关替代料，无row值的则为没有替代料，后续直接根据SKU匹配刊登时间
df_2u_row = df_2u[df_2u['row']!=''].reset_index(drop=True)
df_2u_row = pd.merge(left=df_2u_row, 
                     right=df_listing_2u, 
                     how='left', 
                     left_on='row',
                     right_on='row', 
                     suffixes=('','_x'))
df_2u_row['2u'] = [pd.NaT if pd.isna(df_2u_row.loc[i,'刊登日期']) else df_2u_row.loc[i,'刊登日期'] for i in range(df_2u_row.shape[0])]
df_2u_row = df_2u_row[df_2u_row.columns[:-3]]
# len(df_2u_row)

In [31]:
# 无row值的则为没有替代料，直接根据SKU匹配刊登时间
df_2u_notrow = df_2u[df_2u['row']==''].reset_index(drop=True)
df_2u_notrow = pd.merge(left=df_2u_notrow, 
                        right=df_listing_2u,
                        how='left',
                        left_on='SKU', 
                        right_on='SKU',
                        suffixes=('','_x'))
df_2u_notrow['2u'] = [df_2u_notrow.loc[i,'2u'] if pd.isna(df_2u_notrow.loc[i,'刊登日期']) else df_2u_notrow.loc[i,'刊登日期'] for i in range(df_2u_notrow.shape[0])]
df_2u_notrow = df_2u_notrow[df_2u_notrow.columns[:-3]]
# df_2u_notrow

In [32]:
df_2u_tmp = pd.concat([df_2u_row,df_2u_notrow])
df_2u_tmp = df_2u_tmp.sort_values(by='原始行号').reset_index(drop=True)
# df_2u_tmp

### 3.5 在途匹配US-T(Total)账号信息

In [33]:
# 选择有row值来匹配相关替代料，无row值的则为没有替代料，后续直接根据SKU匹配刊登时间
df_total_row = df_total[df_total['row']!=''].reset_index(drop=True)
df_total_row = pd.merge(left=df_total_row, 
                        right=df_listing_total,
                        how='left', 
                        left_on='row', 
                        right_on='row', 
                        suffixes=('','_x'))
df_total_row['total'] = [pd.NaT if pd.isna(df_total_row.loc[i,'刊登日期']) else df_total_row.loc[i,'刊登日期'] for i in range(df_total_row.shape[0])]
df_total_row = df_total_row[df_total_row.columns[:-3]]
# df_total_row

In [34]:
# 无row值的则为没有替代料，直接根据SKU匹配刊登时间
df_total_notrow = df_total[df_total['row']==''].reset_index(drop=True)
df_total_notrow = pd.merge(left=df_total_notrow, 
                           right=df_listing_total,
                           how='left', left_on='SKU', 
                           right_on='SKU', 
                           suffixes=('','_x'))
df_total_notrow['total'] = [df_total_notrow.loc[i,'total'] if pd.isna(df_total_notrow.loc[i,'刊登日期']) else df_total_notrow.loc[i,'刊登日期'] for i in range(df_total_notrow.shape[0])]
df_total_notrow = df_total_notrow[df_total_notrow.columns[:-3]]
# df_total_notrow

In [35]:
df_total_tmp = pd.concat([df_total_row,df_total_notrow])
df_total_tmp = df_total_tmp.sort_values(by='原始行号').reset_index(drop=True)
# df_total_tmp[pd.notna(df_total_tmp['total'])]
# df_total_tmp

## 3.6将信息匹配回原来的表格里

In [36]:
# 将匹配到的ap账号的时间匹配到原来大账号不为空的总表里
df_progress_transit_tmp2 = df_progress_transit_tmp
df_ap_tmp = df_ap_tmp[['ap', '原始行号']]
df_progress_transit_tmp2 = pd.merge(left=df_progress_transit_tmp2, 
                                    right=df_ap_tmp, 
                                    how='left', 
                                    left_on='原始行号', 
                                    right_on='原始行号', 
                                    suffixes=('','_x')).fillna('')
df_progress_transit_tmp2['ap_x'] = df_progress_transit_tmp2['ap_x'].where(df_progress_transit_tmp2['ap_x'].isna(), '' )
df_progress_transit_tmp2['ap_x'] = df_progress_transit_tmp2['ap_x'].where(pd.isna(df_progress_transit_tmp2['ap_x']), '' )
df_progress_transit_tmp2['ap'] = [df_progress_transit_tmp2.loc[i,'ap_x'] if (df_progress_transit_tmp2.loc[i,'ap_x']!='') else df_progress_transit_tmp2.loc[i,'ap'] for i in range(df_progress_transit_tmp2.shape[0])]
df_progress_transit_tmp2 = df_progress_transit_tmp2[df_progress_transit_tmp2.columns[:-1]]

In [37]:
# 将匹配到的2u账号的时间匹配到原来大账号不为空的总表里
df_2u_tmp = df_2u_tmp[['2u', '原始行号']]
df_progress_transit_tmp2 = pd.merge(left=df_progress_transit_tmp2, right=df_2u_tmp, how='left', left_on='原始行号', right_on='原始行号', suffixes=('','_x')).fillna('')
df_progress_transit_tmp2['2u'] = [df_progress_transit_tmp2.loc[i,'2u_x'] if pd.notna(df_progress_transit_tmp2.loc[i,'2u_x']) else df_progress_transit_tmp2.loc[i,'2u'] for i in range(df_progress_transit_tmp2.shape[0])]
# df_progress_transit_tmp2 = df_progress_transit_tmp2 = df_progress_transit_tmp2[df_progress_transit_tmp2.columns[:-1]]

In [38]:
# 将匹配到的 US-T 账号的时间匹配到原来大账号不为空的总表里
df_total_tmp = df_total_tmp[['total', '原始行号']]
df_progress_transit_tmp2 = pd.merge(left=df_progress_transit_tmp2, 
                                    right=df_total_tmp, how='left', 
                                    left_on='原始行号', 
                                    right_on='原始行号',
                                    suffixes=('','_x')).fillna('')
df_progress_transit_tmp2['total'] = [df_progress_transit_tmp2.loc[i,'total_x'] if pd.notna(df_progress_transit_tmp2.loc[i,'total_x']) else df_progress_transit_tmp2.loc[i,'total'] for i in range(df_progress_transit_tmp2.shape[0])]
df_progress_transit_tmp2 = df_progress_transit_tmp2 = df_progress_transit_tmp2[df_progress_transit_tmp2.columns[:-1]]

In [39]:
df_progress_transit_tmp2 = df_progress_transit_tmp2[df_progress_transit_tmp2.columns[:-2]]

In [40]:
len(df_progress_transit_tmp2)

8052

In [41]:
df_progress_transit_null = df_progress_transit[(pd.isna(df_progress_transit['大账号']))].reset_index(drop=True)
df_progress_transit_tmp3 = pd.concat([df_progress_transit_null, df_progress_transit_tmp2]).sort_values('原始行号').reset_index(drop=True)
df_progress_transit_tmp3 = df_progress_transit_tmp3[['大账号', 'ap', '2u', 'total']]

In [63]:
df_progress_transit_tmp3.to_excel('./ap_2u_total_time_{}.xlsx'.format(datetime.now().strftime("%Y%m%d_%H%M%S")),index=False)

# 4.在途表匹配大账号刊登时间

In [43]:
df_progress_transit_isna = df_progress_transit_tmp_total[df_progress_transit_tmp_total['大账号'].isna()].reset_index(drop=True)
  
# 选择有row值来匹配相关替代料，无row值的则为没有替代料，后续直接根据SKU匹配刊登时间
df_progress_transit_isna_row = df_progress_transit_isna[df_progress_transit_isna['row']!=''].reset_index(drop=True)
df_progress_transit_isna_row = pd.merge(left=df_progress_transit_isna_row, 
                                             right=df_listing_ap, 
                                             how='left', 
                                             left_on='row', 
                                             right_on='row', 
                                             suffixes=('','_x'))
df_progress_transit_isna_row['ap'] = [pd.NaT if pd.isna(df_progress_transit_isna_row.loc[i,'刊登日期']) else df_progress_transit_isna_row.loc[i,'刊登日期'] for i in range(df_progress_transit_isna_row.shape[0])]
df_progress_transit_isna_row = df_progress_transit_isna_row[df_progress_transit_isna_row.columns[:-3]]
# df_progress_transit_isna_row

In [44]:
# 无row值的则为没有替代料，直接根据SKU匹配刊登时间
df_progress_transit_isna_notrow = df_progress_transit_isna[df_progress_transit_isna['row']==''].reset_index(drop=True)
df_progress_transit_isna_notrow = pd.merge(left=df_progress_transit_isna_notrow, 
                                            right=df_listing_ap, 
                                            how='left', 
                                            left_on='SKU',
                                            right_on='SKU', 
                                            suffixes=('','_x'))
df_progress_transit_isna_notrow['ap'] = [df_progress_transit_isna_notrow.loc[i,'ap'] if pd.isna(df_progress_transit_isna_notrow.loc[i,'刊登日期']) else df_progress_transit_isna_notrow.loc[i,'刊登日期'] for i in range(df_progress_transit_isna_notrow.shape[0])]
df_progress_transit_isna_notrow = df_progress_transit_isna_notrow[df_progress_transit_isna_notrow.columns[:-3]]
# df_progress_transit_isna_notrow

In [45]:
df_progress_transit_isna_tmp = pd.concat([df_progress_transit_isna_row,df_progress_transit_isna_notrow])
df_progress_transit_isna_tmp = df_progress_transit_isna_tmp.sort_values(by='原始行号').reset_index(drop=True)
# df_progress_transit_isna_tmp

In [46]:
df_progress_transit_isna_tmp_total = pd.concat([df_progress_transit_isna_tmp,df_progress_transit_tmp]).sort_values(by='原始行号').reset_index(drop=True)
df_progress_transit_isna_tmp_total = df_progress_transit_tmp_total[['大账号', 'ap', '2u', 'total']]

In [64]:
df_progress_transit_isna_tmp_total.to_excel('./大账号为空_{}.xlsx'.format(datetime.now().strftime("%Y%m%d_%H%M%S")), index=False)